# Complete Fragment Processing Workflow

This notebook performs the complete workflow for processing scATAC-seq fragments across multiple species:
1. **Link fragments** to organized folder structure (by cell type)
2. **Liftover fragments** to human genome (hg38) using parallel implementation
3. **Convert fragments** to coverage bigWig files

---

## Setup and Configuration

Define species, auto-detect cell types, and configure file paths.

In [1]:
# Activate the genomes environment for liftover operations
mamba activate genomes

In [2]:
# Define species to process
SPECIES=("Human" "Gorilla" "Chimpanzee" "Bonobo" "Macaque" "Marmoset")

# Base directories
BASE_DIR="/cluster/project/treutlein/USERS/jjans/analysis/adult_intestine/atac"
CHAIN_DIR="${BASE_DIR}/genomes/chains"
MARMOSET_CHAIN_DIR="/cluster/work/treutlein/jjans/data/intestine/nhp_atlas/genomes/chain_files"
CHROM_SIZES="/cluster/home/jjanssens/jjans/analysis/cerebellum/genomes_new/homo_sapiens/hg38.chrom.sizes"

# Chain file mapping
declare -A CHAIN_FILES
CHAIN_FILES["Gorilla"]="${CHAIN_DIR}/gorGor4ToHg38.over.chain.gz"
CHAIN_FILES["Chimpanzee"]="${CHAIN_DIR}/panTro5ToHg38.over.chain.gz"
CHAIN_FILES["Bonobo"]="${CHAIN_DIR}/panPan2ToHg38.over.chain.gz"
CHAIN_FILES["Macaque"]="${CHAIN_DIR}/rheMac10ToHg38.over.chain.gz"
CHAIN_FILES["Marmoset_step1"]="${MARMOSET_CHAIN_DIR}/calJac1ToCalJac4.over.chain"
CHAIN_FILES["Marmoset_step2"]="${MARMOSET_CHAIN_DIR}/calJac4ToHg38.over.chain"

echo "✅ Configuration loaded"

✅ Configuration loaded


### Auto-detect Available Cell Types

Scan all species directories to find available cell types.

In [3]:
# Auto-detect cell types from available fragment files across all species
echo "🔍 Scanning for available cell types across all species..."
echo ""

# Create associative array to store unique cell types
declare -A UNIQUE_CELL_TYPES

# Scan each species directory
for species in "${SPECIES[@]}"; do
    species_dir="${BASE_DIR}/consensus_peak_calling_${species}/pseudobulk_bed_files"
    
    if [[ -d "$species_dir" ]]; then
        echo "📂 Scanning ${species}..."
        
        # Find all .fragments.tsv.gz files and extract cell type names
        while IFS= read -r file; do
            if [[ -n "$file" ]]; then
                # Extract cell type name (remove .fragments.tsv.gz extension)
                cell_type=$(basename "$file" .fragments.tsv.gz)
                UNIQUE_CELL_TYPES["$cell_type"]=1
                echo "   ✓ Found: ${cell_type}"
            fi
        done < <(find "$species_dir" -maxdepth 1 -name "*.fragments.tsv.gz" -type f 2>/dev/null)
    else
        echo "⚠️  Directory not found for ${species}: $species_dir"
    fi
    echo ""
done

# Convert associative array keys to regular array
CELL_TYPES=($(echo "${!UNIQUE_CELL_TYPES[@]}" | tr ' ' '\n' | sort))

# Display detected cell types
echo "================================"
echo "📋 Detected Cell Types (${#CELL_TYPES[@]} total):"
echo "================================"
for cell_type in "${CELL_TYPES[@]}"; do
    echo "  • $cell_type"
done
echo ""

# Create a summary of which cell types are present in which species
echo "================================"
echo "📊 Cell Type × Species Matrix:"
echo "================================"
for cell_type in "${CELL_TYPES[@]}"; do
    echo ""
    echo "📌 ${cell_type}:"
    for species in "${SPECIES[@]}"; do
        file_path="${BASE_DIR}/consensus_peak_calling_${species}/pseudobulk_bed_files/${cell_type}.fragments.tsv.gz"
        if [[ -f "$file_path" ]]; then
            echo "   ✅ ${species}"
        else
            echo "   ❌ ${species} (not available)"
        fi
    done
done

echo ""
echo "✅ Cell type detection completed"

🔍 Scanning for available cell types across all species...

📂 Scanning Human...
   ✓ Found: Adipocytes
   ✓ Found: BEST4+_cells
   ✓ Found: Crypt_Fibroblasts_WNT2B+
   ✓ Found: Colonocytes
   ✓ Found: Enterocytes
   ✓ Found: Specialized_Fibroblasts_VCAM1+
   ✓ Found: Villus_Fibroblasts_WNT5B+
   ✓ Found: T_cells
   ✓ Found: Specialized_Fibroblasts_SYNM+
   ✓ Found: Monocytes
   ✓ Found: Plasma_B_cells
   ✓ Found: Stem_cells
   ✓ Found: Tuft_cells
   ✓ Found: TA_cells
   ✓ Found: Specialized_Fibroblasts_RSPO2_3+
   ✓ Found: Specialized_Fibroblasts_RSPO3+_only
   ✓ Found: Pericytes
   ✓ Found: Specialized_Fibroblasts_KCNN3+
   ✓ Found: ILCs
   ✓ Found: Lymphatic_ECs
   ✓ Found: Goblet_cells
   ✓ Found: Macrophages
   ✓ Found: ICCs
   ✓ Found: Myofibroblasts
   ✓ Found: Naive_B_cells
   ✓ Found: EECs
   ✓ Found: Enteric_glia
   ✓ Found: Mast_cells
   ✓ Found: ECs
   ✓ Found: Enteric_neurons
   ✓ Found: MUC6+_cells
   ✓ Found: Paneth_cells

📂 Scanning Gorilla...
   ✓ Found: Specialized_Fibr

---
## Step 3: Convert Fragments to BigWig Coverage Files

Convert lifted fragment files to coverage bigWig format for visualization.

**Note:** Requires `scatac_fragment_tools` to be installed.

In [4]:
# Activate the scatac_fragment_tools environment
mamba activate scatac_fragment_tools

In [5]:
# Create output directories for bigWig files (organized by cell type)
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    mkdir -p "bigwigs/${cell_type_lower}"
    echo "📁 Created directory: bigwigs/${cell_type_lower}"
done

echo "✅ BigWig output directories created"

📁 Created directory: bigwigs/adipocytes
📁 Created directory: bigwigs/best4+_cells
📁 Created directory: bigwigs/colonocytes
📁 Created directory: bigwigs/crypt_fibroblasts_wnt2b+
📁 Created directory: bigwigs/ecs
📁 Created directory: bigwigs/eecs
📁 Created directory: bigwigs/enteric_glia
📁 Created directory: bigwigs/enteric_neurons
📁 Created directory: bigwigs/enterocytes
📁 Created directory: bigwigs/eosinophils
📁 Created directory: bigwigs/goblet-like_cells
📁 Created directory: bigwigs/goblet_cells
📁 Created directory: bigwigs/iccs
📁 Created directory: bigwigs/ilcs
📁 Created directory: bigwigs/lymphatic_ecs
📁 Created directory: bigwigs/marco+_lymphatic_ecs
📁 Created directory: bigwigs/muc6+_cells
📁 Created directory: bigwigs/macrophages
📁 Created directory: bigwigs/mast_cells
📁 Created directory: bigwigs/mesothelial_cells
📁 Created directory: bigwigs/monocytes
📁 Created directory: bigwigs/myofibroblasts
📁 Created directory: bigwigs/naive_b_cells
📁 Created directory: bigwigs/neutrophils
📁

In [ ]:
# Convert fragments to bigWig for all species and cell types
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    
    for species in "${SPECIES[@]}"; do
        species_lower=$(echo "$species" | tr '[:upper:]' '[:lower:]')
        
        # Determine input file name based on species
        if [[ "$species" == "Human" ]]; then
            # Human uses direct link (no _par suffix)
            input_frag="lifted_fragments/${cell_type_lower}/${species_lower}_${cell_type_lower}.hg38.fragments.tsv.gz"
        else
            # Other species use parallel liftover output (_par suffix)
            input_frag="lifted_fragments/${cell_type_lower}/${species_lower}_${cell_type_lower}.hg38.fragments_par.tsv.gz"
        fi
        
        output_bw="bigwigs/${cell_type_lower}/${species_lower}_${cell_type_lower}.hg38.cov.bw"
        
        if [[ -f "$input_frag" && -f "$CHROM_SIZES" ]]; then
            echo "🎯 Converting ${species} ${cell_type} to bigWig..."
            scatac_fragment_tools bigwig \
                -i "$input_frag" \
                -c "$CHROM_SIZES" \
                -o "$output_bw" \
                -n
            echo "✅ Done: $output_bw"
        else
            echo "⚠️  Missing input for ${species} ${cell_type} — skipping"
        fi
        echo ""
    done
done

echo "✅ All bigWig conversions completed"

🎯 Converting Human Adipocytes to bigWig...
✅ Done: bigwigs/adipocytes/human_adipocytes.hg38.cov.bw

🎯 Converting Gorilla Adipocytes to bigWig...
✅ Done: bigwigs/adipocytes/gorilla_adipocytes.hg38.cov.bw

🎯 Converting Chimpanzee Adipocytes to bigWig...
✅ Done: bigwigs/adipocytes/chimpanzee_adipocytes.hg38.cov.bw

🎯 Converting Bonobo Adipocytes to bigWig...
✅ Done: bigwigs/adipocytes/bonobo_adipocytes.hg38.cov.bw

🎯 Converting Macaque Adipocytes to bigWig...
✅ Done: bigwigs/adipocytes/macaque_adipocytes.hg38.cov.bw

🎯 Converting Marmoset Adipocytes to bigWig...
✅ Done: bigwigs/adipocytes/marmoset_adipocytes.hg38.cov.bw

🎯 Converting Human BEST4+_cells to bigWig...
✅ Done: bigwigs/best4+_cells/human_best4+_cells.hg38.cov.bw

🎯 Converting Gorilla BEST4+_cells to bigWig...
✅ Done: bigwigs/best4+_cells/gorilla_best4+_cells.hg38.cov.bw

🎯 Converting Chimpanzee BEST4+_cells to bigWig...
✅ Done: bigwigs/best4+_cells/chimpanzee_best4+_cells.hg38.cov.bw

🎯 Converting Bonobo BEST4+_cells to bigWig

In [ ]:
# Verify bigWig files
for cell_type in "${CELL_TYPES[@]}"; do
    cell_type_lower=$(echo "$cell_type" | tr '[:upper:]' '[:lower:]')
    echo "\n📂 BigWig files for ${cell_type}:"
    ls -lh "bigwigs/${cell_type_lower}/"
done

---
## Summary

### Output Structure

```
fragment_files/
└── enterocytes/
    ├── human_enterocytes.fragments.tsv.gz
    ├── gorilla_enterocytes.fragments.tsv.gz
    ├── chimpanzee_enterocytes.fragments.tsv.gz
    ├── bonobo_enterocytes.fragments.tsv.gz
    ├── macaque_enterocytes.fragments.tsv.gz
    └── marmoset_enterocytes.fragments.tsv.gz

lifted_fragments/
└── enterocytes/
    ├── human_enterocytes.hg38.fragments.tsv.gz
    ├── gorilla_enterocytes.hg38.fragments_par.tsv.gz
    ├── chimpanzee_enterocytes.hg38.fragments_par.tsv.gz
    ├── bonobo_enterocytes.hg38.fragments_par.tsv.gz
    ├── macaque_enterocytes.hg38.fragments_par.tsv.gz
    └── marmoset_enterocytes.hg38.fragments_par.tsv.gz

bigwigs/
└── enterocytes/
    ├── human_enterocytes.hg38.cov.bw
    ├── gorilla_enterocytes.hg38.cov.bw
    ├── chimpanzee_enterocytes.hg38.cov.bw
    ├── bonobo_enterocytes.hg38.cov.bw
    ├── macaque_enterocytes.hg38.cov.bw
    └── marmoset_enterocytes.hg38.cov.bw
```

### Processing Notes

- **Human**: No liftover needed (already hg38), direct symlink created
- **Standard species** (Gorilla, Chimpanzee, Bonobo, Macaque): Single-step liftover with 30 CPUs
- **Marmoset**: Two-step liftover (calJac1 → calJac4 → hg38) with 30 CPUs
- All outputs organized by cell type for easy access and management
- BigWig files use normalized coverage (`-n` flag) for cross-sample comparison